In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import joblib
import time
import warnings 
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(executable_path = 'C:\\Users\\nakul\\chrome_driver\\chromedriver.exe')

### Extracting countries yearwise data

In [3]:
driver.get('https://www.olympedia.org/statistics/medal/country')

In [4]:
s = driver.find_element(by='id' ,value = 'competition_type_ids_2')

In [5]:
s.click()

In [6]:
d = driver.find_element(by='id' ,value = 'season_id')
select = Select(d)

In [7]:
select.select_by_visible_text('Summer')

In [8]:
data = pd.read_html(driver.page_source)[0]
data.rename(columns={'NOC.1':'country_code'},inplace=True)   

In [9]:
dic = dict(zip(data.country_code,data.NOC))

data.rename(columns = {'NOC' : 'Country'},inplace=True)
data.drop(columns = ['country_code'],inplace=True)
data['year'] = '1896 - 2016'
df = data

In [10]:
df.head()

,Country,Gold,Silver,Bronze,Total,year
0,United States,1206,982,860,3048,1896 - 2016
1,Soviet Union,473,376,355,1204,1896 - 2016
2,Germany,371,407,403,1181,1896 - 2016
3,People's Republic of China,370,268,230,868,1896 - 2016
4,Great Britain,317,348,353,1018,1896 - 2016


In [11]:
years = list(range(1896,2017,4))
years.remove(1916)
years.remove(1940)
years.remove(1944)
years = [str(i) for i in years]
yearwise_data = pd.DataFrame()

for i in years:
    d = driver.find_element(by='id' ,value = 'edition_select')
    select = Select(d)
    select.select_by_visible_text(i)
    time.sleep(20)
    data = pd.read_html(driver.page_source,encoding='utf-8')[0]
    data.rename(columns={'NOC.1':'country_code'},inplace=True)
    data['year'] = int(i)
    yearwise_data = pd.concat([yearwise_data,data],axis=0)
    print('Done for the year = ',i)

Done for the year =  1896
Done for the year =  1900
Done for the year =  1904
Done for the year =  1908
Done for the year =  1912
Done for the year =  1920
Done for the year =  1924
Done for the year =  1928
Done for the year =  1932
Done for the year =  1936
Done for the year =  1948
Done for the year =  1952
Done for the year =  1956
Done for the year =  1960
Done for the year =  1964
Done for the year =  1968
Done for the year =  1972
Done for the year =  1976
Done for the year =  1980
Done for the year =  1984
Done for the year =  1988
Done for the year =  1992
Done for the year =  1996
Done for the year =  2000
Done for the year =  2004
Done for the year =  2008
Done for the year =  2012
Done for the year =  2016


In [12]:
yearwise_data.rename(columns = {'NOC' : 'Country'},inplace=True)
yearwise_data.drop(columns = ['country_code'],inplace=True)

countries = sorted(yearwise_data['Country'].unique().tolist())
years = sorted(yearwise_data['year'].unique().tolist())

countries.insert(0,'Overall')
years.insert(0,'Overall')

joblib.dump(countries,'countries.joblib')
joblib.dump(years,'years.joblib')

['years.joblib']

In [13]:
yearwise_data.head()

,Country,Gold,Silver,Bronze,Total,year
0,United States,1088,858,763,2709,1896
1,Soviet Union,395,319,296,1010,1896
2,People's Republic of China,344,228,199,771,1896
3,Great Britain,304,342,334,980,1896
4,France,262,283,322,867,1896


In [14]:
df = pd.concat([df,yearwise_data],axis=0)
df.to_csv('Country_data.csv',index=False)

In [15]:
df.head()

,Country,Gold,Silver,Bronze,Total,year
0,United States,1206,982,860,3048,1896 - 2016
1,Soviet Union,473,376,355,1204,1896 - 2016
2,Germany,371,407,403,1181,1896 - 2016
3,People's Republic of China,370,268,230,868,1896 - 2016
4,Great Britain,317,348,353,1018,1896 - 2016


### Extracting athletes yearwise data

In [16]:
driver.get('http://www.olympedia.org/statistics/medal/athlete')

In [17]:
s = driver.find_element(by='id' ,value = 'competition_type_ids_2')

In [18]:
s.click()

In [19]:
d = driver.find_element(by='id' ,value = 'season_id')
select = Select(d)

In [20]:
select.select_by_visible_text('Summer')

In [21]:
d = driver.find_element(by='name' ,value = 'limit')
select = Select(d)

In [22]:
select.select_by_visible_text('Show all')

In [23]:
data = pd.read_html(driver.page_source)[0]
data.rename(columns={'Nat':'country_code'},inplace=True)   

In [24]:
data.head()

,Athlete,country_code,Gold,Silver,Bronze,Total
0,Michael Phelps,USA,23,3,2,28
1,Larisa Latynina,URS UKR,9,5,4,18
2,Paavo Nurmi,FIN,9,3,0,12
3,Mark Spitz,USA,9,1,1,11
4,Carl Lewis,USA,9,1,0,10


In [25]:
def preprocess(df):
    temp = df.country_code.str.split(' ',expand=True)
    country = temp[0].values.tolist()
    df['country_code'] = country
    
    for i in range(temp.shape[1]-1):
        temp_data = temp.dropna(subset=[i+1])
        idx = temp_data.index.tolist()
        data = temp_data[i+1].values.tolist()
        d = df.iloc[idx,:]
        d['country_code'] = data
        df = pd.concat([df,d],axis=0)
        
    return df

In [26]:
data = preprocess(data)
data = data.reset_index(drop=True)
idx = data.country_code.values.tolist()

d = dic
temp = []
for i in range(data.shape[0]):
    temp.append(d[idx[i]])
    
data['country_code'] = temp
data = data.rename(columns={'country_code':'Country'})
data['year'] = '1896 - 2016'
df = data

In [27]:
df.head()

,Athlete,Country,Gold,Silver,Bronze,Total,year
0,Michael Phelps,United States,23,3,2,28,1896 - 2016
1,Larisa Latynina,Soviet Union,9,5,4,18,1896 - 2016
2,Paavo Nurmi,Finland,9,3,0,12,1896 - 2016
3,Mark Spitz,United States,9,1,1,11,1896 - 2016
4,Carl Lewis,United States,9,1,0,10,1896 - 2016


In [28]:
years = list(range(1896,2017,4))
years.remove(1916)
years.remove(1940)
years.remove(1944)
years = [str(i) for i in years]
yearwise_athletes_data = pd.DataFrame()

for i in years:
    d = driver.find_element(by='id' ,value = 'edition_select')
    select = Select(d)
    select.select_by_visible_text(i)
    time.sleep(20)
    data = pd.read_html(driver.page_source,encoding='utf-8')[0]
    data.rename(columns={'Nat':'country_code'},inplace=True)
    data['year'] = int(i)
    yearwise_athletes_data = pd.concat([yearwise_athletes_data,data],axis=0)
    print('Done for the year = ',i)  

Done for the year =  1896
Done for the year =  1900
Done for the year =  1904
Done for the year =  1908
Done for the year =  1912
Done for the year =  1920
Done for the year =  1924
Done for the year =  1928
Done for the year =  1932
Done for the year =  1936
Done for the year =  1948
Done for the year =  1952
Done for the year =  1956
Done for the year =  1960
Done for the year =  1964
Done for the year =  1968
Done for the year =  1972
Done for the year =  1976
Done for the year =  1980
Done for the year =  1984
Done for the year =  1988
Done for the year =  1992
Done for the year =  1996
Done for the year =  2000
Done for the year =  2004
Done for the year =  2008
Done for the year =  2012
Done for the year =  2016


In [29]:
yearwise_athletes_data.head()

,Athlete,country_code,Gold,Silver,Bronze,Total,year
0,Michael Phelps,USA,23,3,2,28,1896
1,Leonidas,RHD,12,0,0,12,1896
2,Herodoros,MEG,11,0,0,11,1896
3,Ray Ewry,USA,10,0,0,10,1896
4,Larisa Latynina,URS UKR,9,5,4,18,1896


In [30]:
yearwise_athletes_data = preprocess(yearwise_athletes_data)

idx = yearwise_athletes_data.country_code.values.tolist()
d = dic
d['SAA'] = 'Saar'
d['ALB'] = 'Albania'
temp = []
for i in range(yearwise_athletes_data.shape[0]):
    temp.append(d[idx[i]])
    
yearwise_athletes_data['country_code'] = temp
yearwise_athletes_data = yearwise_athletes_data.rename(columns={'country_code':'Country'})

unique_athletes = sorted(yearwise_athletes_data.Athlete.unique().tolist())
unique_countries = sorted(yearwise_athletes_data.Country.unique().tolist())

unique_athletes.insert(0,'Overall')
unique_countries.insert(0,'Overall')

joblib.dump(unique_countries,'unique_countries.joblib')
joblib.dump(unique_athletes,'unique_athletes.joblib')

df = pd.concat([df,yearwise_athletes_data],axis=0)
df.to_csv('athletes_data.csv',index=False)

In [31]:
df.head()

,Athlete,Country,Gold,Silver,Bronze,Total,year
0,Michael Phelps,United States,23,3,2,28,1896 - 2016
1,Larisa Latynina,Soviet Union,9,5,4,18,1896 - 2016
2,Paavo Nurmi,Finland,9,3,0,12,1896 - 2016
3,Mark Spitz,United States,9,1,1,11,1896 - 2016
4,Carl Lewis,United States,9,1,0,10,1896 - 2016
